In [67]:
import pandas as pd



## VALUATIONS

In [69]:
# Read valuations
valuations_path = 'datasets/player_valuations.csv'

# Read the CSV file into a pandas DataFrame
df_val = pd.read_csv(valuations_path)

#drop unesseccary columns
df_val = df_val.drop(columns=["datetime", "dateweek"])


## PLAYER STATS

In [122]:
# Read stats
"""
Columns
appearance_id,game_id,player_id,player_club_id,player_current_club_id,date,player_name,competition_id,yellow_cards,red_cards,goals,assists,minutes_played
"""
stats_path = 'datasets/appearances.csv'

# Read the CSV file into a pandas DataFrame
df_stats = pd.read_csv(stats_path)

#drop unesseccary columns
#TBD

## CATEGORICAL DATA

In [174]:
"""
CATEGORICAL DATA HANDLING
Columns
player_id,first_name,last_name,name,last_season,current_club_id,player_code,country_of_birth,city_of_birth,country_of_citizenship,date_of_birth,sub_position,position,foot,height_in_cm,contract_expiration_date,agent_name,image_url,url,current_club_domestic_competition_id,current_club_name,market_value_in_eur,highest_market_value_in_eur
"""

# Read player categorical data
categorical_path = 'datasets/players.csv'

df_categorical = pd.read_csv(categorical_path)

#drop redundant columns
#comment: dropping sub-position as all players only have one
df_cat = df_categorical.drop(
    columns=[
        "first_name", "last_name", "name", "player_code", "city_of_birth", "country_of_citizenship", "image_url", "url", "highest_market_value_in_eur", "current_club_name",
        "agent_name", "current_club_domestic_competition_id", "market_value_in_eur", "current_club_domestic_competition_id", "current_club_id", "last_season", "sub_position"
        ])


#Change date of birth to age
df_cat['date_of_birth'] = pd.to_datetime(df_cat['date_of_birth'])
def calculate_age(birthdate):
    today = pd.to_datetime('today')
    age = today.year - birthdate.year - ((today.month, today.day) < (birthdate.month, birthdate.day))
    return age

df_cat['age'] = df_cat['date_of_birth'].apply(calculate_age)

#date of birth now redundant
df_cat = df_cat.drop(columns=["date_of_birth"])


# Fill NaN values in 'height_in_cm' with the mean
mean_value = df_cat['height_in_cm'].mean()
df_cat['height_in_cm'].fillna(mean_value, inplace=True)

#Fill NaN values in col Foot with 'right'
most_frequent_value = df_cat['foot'].mode().iloc[0]
df_cat['foot'].fillna(most_frequent_value, inplace=True)

## CLUB GAMES

In [188]:
"""
Club games data handling
Columns: game_id,club_id,own_goals,own_position,own_manager_name,opponent_id,opponent_goals,opponent_position,opponent_manager_name,hosting,is_win
"""

# Read player categorical data
club_games_path = 'datasets/club_games.csv'

df_club_games = pd.read_csv(club_games_path)


#aggregate stats for last season
df_aggregated_club_games= df_club_games.groupby('club_id').agg({
    'is_win': 'sum'
}).reset_index()


df_aggregated_club_games

,club_id,is_win
0,1,9
1,2,14
2,3,101
3,4,33
4,5,271
...,...,...
2681,112751,1
2682,112752,0
2683,112753,1
2684,112754,0


## CONSTANTS

In [156]:
#Get latest season end year
latest_date = df_val['date'].max()
latest_year = int(latest_date[0:4])
latest_year

2023

## EDA / DATA HANDLING

Fix contract_expiration

In [176]:
#Change contract expiration date to see how many months they have left at end of season
df_cat['contract_expiration_date'] = pd.to_datetime(df_cat['contract_expiration_date'])

current_date = pd.to_datetime(str(latest_year) + "-07-01") #get latest date we are looking at

# Calculate the difference in months between the contract expiration date and the current date
df_cat['months_left'] = (df_cat['contract_expiration_date'] - current_date).fillna(pd.Timedelta(days=0)).astype('timedelta64[M]').astype(int)

#contract expiration now redundant
df_cat = df_cat.drop(columns=["contract_expiration_date"])

# Set negative values to zero because there will be some outgoing contracts at end of seasons
df_cat['months_left'] = df_cat['months_left'].clip(lower=0)

Only use last season and top 5 leagues in the world

In [177]:
#Select only specific leagues
competition_column = 'player_club_domestic_competition_id'
competitions = ["IT1", "GB1", "FR1", "L1", "ES1"]

#Take values from only last season
date_column = "date"
max_date_last_season = str(latest_year) + "-07-01"
min_date_last_season = str(latest_year-1) + "-07-01"

# Create a new DataFrame containing only rows where the 'column_name' matches the 'desired_value'
df_val_last_season = df_val[(df_val[competition_column].isin(competitions)) 
                                 & (df_val[date_column] >= min_date_last_season)
                                 & (df_val[date_column] <= max_date_last_season)]

df_stats_last_season = df_stats[(df_stats[date_column] >= min_date_last_season)
                                 & (df_stats[date_column] <= max_date_last_season)]

#Sort based on date
df_sorted = df_val_last_season.sort_values(by=['player_id', 'date'], ascending=[True, False])

# Drop duplicates, keeping the first entry for each 'playerID'
df_latest_valuations_last_season = df_sorted.drop_duplicates(subset='player_id', keep='first')

In [179]:
df_stats_last_season

,appearance_id,game_id,player_id,player_club_id,player_current_club_id,date,player_name,competition_id,yellow_cards,red_cards,goals,assists,minutes_played
1304138,3836618_504518,3836618,504518,3461,873,2022-07-05,Kofi Balmer,ECLQ,1,0,0,0,90
1304139,3837917_424019,3837917,424019,3471,465,2022-07-05,Danny Finlayson,CLQ,1,0,0,0,90
1304140,3837919_484873,3837919,484873,10625,16704,2022-07-05,Kady Borges,CLQ,0,0,0,0,83
1304141,3837920_627034,3837920,627034,4644,11688,2022-07-05,Faustin Senghor,CLQ,0,0,0,0,90
1304142,3845948_230746,3845948,230746,496,589,2022-07-05,Erdal Rakip,CLQ,1,0,0,0,80
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1447575,4077919_55356,4077919,55356,16704,1083,2023-06-11,Aleksey Ionov,RUP,0,0,0,0,45
1447576,4077919_617520,4077919,617520,16704,16704,2023-06-11,Stanislav Agkatsev,RUP,0,0,0,0,90
1447577,4077919_666616,4077919,666616,16704,16704,2023-06-11,Olakunle Olusegun,RUP,0,0,0,0,90
1447578,4077919_70114,4077919,70114,2410,2410,2023-06-11,Anton Zabolotnyi,RUP,0,0,0,0,27


Aggregate stats for each player over the specific season

In [180]:
#aggregate stats for last season
df_aggregated_stats_last_season = df_stats_last_season.groupby('player_id').agg({
    'yellow_cards': 'sum',
    'red_cards': 'sum',
    'goals': 'sum',
    'assists': 'sum',
    'minutes_played': 'sum',
    'player_name': 'first' 
}).reset_index()

In [152]:
"""
STATS DATA HANDLING
"""

#df_aggregated_stats_last_season = df_aggregated_stats_last_season.drop(columns=["yellow_cards"])

Merge all dataframes together

In [181]:
#Merge all dataframes together
df_val_and_stats_last_season = pd.merge(df_latest_valuations_last_season, df_aggregated_stats_last_season, on='player_id', how='inner')
df_final_last_season = pd.merge(df_val_and_stats_last_season, df_cat, on='player_id', how="inner")


#To add from club: 
#Antal vinster, titlar, 


In [182]:
df_final_last_season

,player_id,date,market_value_in_eur,current_club_id,player_club_domestic_competition_id,yellow_cards,red_cards,goals,assists,minutes_played,player_name,country_of_birth,position,foot,height_in_cm,age,months_left
0,3333,2023-06-20,1500000,1237,GB1,3,0,0,2,1285,James Milner,England,Midfield,right,175.000000,37.0,11
1,3455,2023-06-15,2000000,5,IT1,0,0,1,0,144,Zlatan Ibrahimović,Sweden,Attack,right,195.000000,42.0,0
2,7161,2023-06-22,13000000,15,L1,2,0,14,12,2814,Jonas Hofmann,Germany,Attack,right,176.000000,31.0,47
3,7663,2023-06-13,1500000,150,ES1,3,0,1,2,938,Joaquín,Spain,Attack,right,181.000000,42.0,0
4,7825,2023-06-13,700000,1050,ES1,2,0,0,0,2460,Pepe Reina,Spain,Goalkeeper,right,188.000000,41.0,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2777,1053240,2023-06-27,900000,1147,FR1,3,0,1,0,490,Moussa Soumano,France,Attack,right,182.240244,18.0,35
2778,1056655,2023-06-30,25000,237,ES1,0,0,0,0,3,Rubén Quintanilla,Spain,Midfield,right,182.240244,21.0,0
2779,1087022,2023-06-27,300000,969,FR1,0,0,0,0,35,Serigne Faye,Senegal,Attack,right,186.000000,19.0,11
2780,1101641,2023-06-27,250000,1420,FR1,0,0,0,0,550,Justin-Noël Kalumba,France,Attack,right,182.240244,18.0,35
